In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
df_schools = pd.read_csv("Masterlist of Schools.csv", index_col="school.id")

df_location = pd.read_csv("Schools Location Data.csv", 
                          encoding = "latin-1", 
                          index_col="School ID",
                          usecols=["School ID", "Enrolment", "Latitude", "Longitude"])

df_rooms = pd.read_csv('Rooms data.csv', index_col="School ID")

df_teachers = pd.read_csv("Teachers data.csv", index_col="school.id")

df_elementary = pd.read_csv("Enrollment Master Data_2015_E.csv")[:-1].astype(int).set_index("School ID")

df_secondary = (pd.read_csv('Enrollment Master Data_2015_S.csv')[:-1]
                  .replace(",", "", regex=True)
                  .astype(int)
                  .replace("SPED NG Male", "SPED NG Male SS")
                  .replace("SPED NG Female", "SPED NG Female SS")
                  .set_index("School ID"))

df_mooe = (pd.read_csv('MOOE data.csv', index_col="school.id", usecols=["school.id", " school.mooe "])
             .replace(",", "", regex=True).astype(float))

In [ ]:
df_all = pd.concat([df_schools, df_location, df_rooms, df_teachers, df_elementary, df_secondary, df_mooe], axis=1)
df_all

In [ ]:
df_all.shape

In [ ]:
df_all.isna().sum()

In [ ]:
df_all[df_all.index.duplicated(keep=False)]

In [ ]:
df_all.columns

Let's try to explore the data for both elementary and secondary school

In [ ]:
# Obtain all numeric features + school.classification
df_numeric = df_all[['rooms.standard.academic', 'rooms.standard.unused',
       'rooms.nonstandard.academic', 'rooms.nonstandard.unused',
       'teachers.instructor', 'teachers.mobile', 'teachers.regular',
       'teachers.sped','Enrolment', ' school.mooe ', 'school.classification']]
df_numeric

In [ ]:
# Combine all rooms and all teachers
df_numeric["rooms_total"] = (df_numeric['rooms.standard.academic'] + 
                             df_numeric['rooms.standard.unused'] + 
                             df_numeric['rooms.nonstandard.academic'] + 
                             df_numeric['rooms.nonstandard.unused'])

df_numeric["teachers_total"] = (df_numeric['teachers.instructor'] + 
                             df_numeric['teachers.mobile'] + 
                             df_numeric['teachers.regular'] + 
                             df_numeric['teachers.sped'])
df_final = df_numeric.iloc[:, 8:]
df_final

Let's find the missing values

In [ ]:
df_final.isna().sum()

In [ ]:
df_final_clean = df_final.dropna()
print(df_final_clean.shape)
df_final.isna().sum()

In [ ]:
index_name = (df_final_clean[(df_final_clean["Enrolment"]==0.0) | 
                             (df_final_clean["teachers_total"]==0.0) |
                             (df_final_clean["rooms_total"]==0.0)].index)
index_name

In [ ]:
df_final_clean = df_final_clean.drop(index_name)
df_final_clean.shape[0]

In [ ]:
dropped_rows = df_final.shape[0] - df_final_clean.shape[0]
dropped_rows

In [ ]:
dropped_rows/df_final.shape[0]*100

In [ ]:
df_ratio_ss = df_final_clean[df_final_clean["school.classification"]=='Secondary'].drop(columns=["school.classification"])

In [ ]:
melted = df_ratio_ss.melt()

sns.boxplot(melted.variable, melted.value)

In [ ]:
# Insert code here
# Removing (statistical) outliers for Amount
Q1 = df_ratio_ss[' school.mooe '].quantile(0.05)
Q3 = df_ratio_ss[' school.mooe '].quantile(0.95)
IQR = Q3 - Q1
df_ratio_ss = (df_ratio_ss[(df_ratio_ss[' school.mooe '] >= Q1 - 1.5*IQR) & 
                           (df_ratio_ss[' school.mooe '] <= Q3 + 1.5*IQR)])


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

teacher = df_ratio_ss.teachers_total
room = df_ratio_ss.rooms_total
budget = df_ratio_ss[' school.mooe ']
ax.scatter(teacher, room, budget, s=5)

ax.set_xlabel('Teacher')
ax.set_ylabel('Room')
ax.set_zlabel('Budget')

plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = __________________
df_scaled_ss = scaler.______________(df_ratio_ss)

In [ ]:
print(df_scaled_ss.min())
print(df_scaled_ss.max())

In [ ]:
from sklearn.cluster import KMeans

model = __________
model._________(df_scaled_ss)
cluster_labels = model.____________(df_scaled_ss)   


In [ ]:
from sklearn.cluster import KMeans

ssd = []
range_n_clusters = list(np.arange(1,16))

for num_clusters in ____________:
    kmeans = ___________________
    kmeans.______(df_scaled_ss)
    
    ssd.append(____________)
    
# plot the SSDs for each n_clusters
plt.plot(ssd)

In [ ]:
kmeans = _____________________
kmeans.___________(df_scaled_ss)
cluster_labels = kmeans.__________(df_scaled_ss)   

df_ratio_ss['Cluster_Labels'] = cluster_labels
df_ratio_ss['Cluster_Labels'].value_counts()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
range_n_clusters = list(np.arange(2,9))

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = ___________________
    kmeans._________(df_scaled_ss)
    
    cluster_labels = kmeans.__________(df_scaled_ss)
    
    # silhouette score
    silhouette_avg = __________(df_scaled_ss, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [ ]:
mergings = linkage(df_scaled_ss, method="single", metric='euclidean')

dendrogram(mergings, p=10, truncate_mode="level", leaf_rotation=90)
plt.show()

In [ ]:
mergings = linkage(df_scaled_ss, method="complete", metric='euclidean')
dendrogram(mergings, p=10, truncate_mode="level")
plt.show()

In [ ]:
mergings = linkage(df_scaled_ss, method="average", metric='euclidean')
dendrogram(mergings, p=10, truncate_mode="level")
plt.show()

In [ ]:
cluster_labels = cut_tree(mergings, n_clusters=3).reshape(-1, )

In [ ]:
df_ratio_ss['Cluster_Labels'] = cluster_labels

In [ ]:
df_ratio_ss['Cluster_Labels'].value_counts()

In [ ]:
df_ratio_ss[df_ratio_ss['Cluster_Labels']==0]

In [ ]:
df_ratio_ss[df_ratio_ss['Cluster_Labels']==1]

In [ ]:
df_ratio_ss[df_ratio_ss['Cluster_Labels']==2]

In [ ]:
from sklearn.cluster import DBSCAN

dbs = _______________
dbs_labels = dbs.__________(df_scaled_ss)   

df_ratio_ss['DBS_Labels'] = dbs_labels
df_ratio_ss['DBS_Labels'].value_counts()


In [ ]:
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator

nearest_neighbors = NearestNeighbors(n_neighbors=11)
neighbors = nearest_neighbors.fit(df_scaled_ss)
distances, indices = neighbors.kneighbors(df_scaled_ss)
distances = np.sort(distances[:, 10], axis=0)

i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')

fig = plt.figure(figsize=(5, 5))
knee.plot_knee()
plt.xlabel("Points")
plt.ylabel("Distance")

print(distances[knee.knee])

## Try it yourself

- Does it makes sense to use the raw numbers as features? What can be an alternative to the raw numbers?
- Are there outliers in the other features as well?
- Try exploring different number of clusters. How many clusters can you characterize?
- Try playing with different hierarchical methods. Which gives the best result?
- Try playing with the eps. Can you still create a better model?

## Interpreting clusters

In [ ]:
import seaborn as sns
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,8))

sns.boxplot(x="Cluster_Labels", y="Enrolment", data=df_ratio_ss, ax=axes[0,0])
axes[0,0].set_title("Enrolment", fontsize=16)

sns.boxplot(x=df_ratio_ss.Cluster_Labels, y=df_ratio_ss[" school.mooe "], ax=axes[0,1])
axes[0,1].set_title("MOOE", fontsize=16)

sns.boxplot(x=df_ratio_ss.Cluster_Labels, y=df_ratio_ss.rooms_total, ax=axes[1,0])
axes[1,0].set_title("Rooms", fontsize=16)

sns.boxplot(x=df_ratio_ss.Cluster_Labels, y=df_ratio_ss.teachers_total, ax=axes[1,1])
axes[1,1].set_title("Teachers", fontsize=16)

plt.tight_layout()
plt.show();

#### Radar Chart

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_minmax = scaler.fit_transform(df_ratio_ss)

df_minmax = pd.DataFrame(df_minmax, index=df_ratio_ss.index, columns=df_ratio_ss.columns)

df_minmax['Cluster_Labels'] = cluster_labels

df_clusters = df_minmax.set_index("Cluster_Labels")
df_clusters = df_clusters.groupby("Cluster_Labels").mean().reset_index()
df_clusters

In [ ]:
from math import pi
def make_spider(row, title, color):
 
    # number of variable
    categories=list(df_clusters)[1:]
    N = len(categories)
 
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
 
    # Initialise the spider plot
    ax = plt.subplot(3,3,row+1, polar=True )
 
    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 3.5)
    ax.set_theta_direction(-1)
    
    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='grey', size=8)
 
    # Draw ylabels
    ax.set_rlabel_position(0)
#     plt.yticks([-2, -1, 0, 1, 2], [-2,-1, 0, 1, 2], color="grey", size=7) #for sscaled
#     plt.ylim(-2.5,2.5)
    plt.yticks([-0.25, 0, 0.25, 0.5, 0.75, 1], [-0.25, 0, 0.25, 0.5,0.75, 1], color="grey", size=7) #formmscaled
    plt.ylim(-0.25,1)

    # Ind1
    values=df_clusters.loc[row].drop('Cluster_Labels').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)
 
    # Add a title
    plt.title(title, size=14, color=color, y=1.1)
 
    

In [ ]:
my_dpi=100
plt.figure(figsize=(1000/my_dpi, 1000/my_dpi), dpi=my_dpi)
plt.subplots_adjust(hspace=0.5)

# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df_clusters.index))

for row in range(0, len(df_clusters.index)):
    make_spider(row=row, 
                title='Segment '+(df_clusters['Cluster_Labels'][row]).astype(str), 
                color=my_palette(row))